# DH Algorithm

Developed by Sanjoy **D**asgupta and Daniel **H**su, the DH algorithm operates on a hierarchical clustering. Iteratively, it works its way through the hierarchical tree and (in loose terms) estimates the error if the subsequent tree (pruning) were assigned the majority label to all the nodes. 

This implementation of DH is adapted from Haotian Teng's implementation, found at https://github.com/haotianteng/DH.

In [ ]:
import numpy as np
from numpy.random import choice
from typing import List, Tuple, Union, Iterable
from scipy.cluster.hierarchy import to_tree
from scipy.cluster.hierarchy import ClusterNode